In [35]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers

In [37]:
# Command to convert .log.labeled files to .csv
# tr '\t' ',' < "(name).log.labeled" | cut -d ',' -f 1-22 > "(name).csv"

# Command to change directory
# cd /mnt/(path name)

# Replace first 7 lines of .csv file (until #types) with the header below
# ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,label,detailed-label

import os
import pandas as pd

folder_path = 'D:\AI Training\ML Model\Test\CSV'
dataframes = []

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path, sep=',')
        dataframes.append(df)

combined_data = pd.concat(dataframes, ignore_index=True)

print(combined_data)

ParserError: Error tokenizing data. C error: Expected 3 fields in line 7, saw 22
